In [9]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")


# Conéctese a OpenAI, Anthropic y Google; comente las líneas de Claude o Google si no las está usando

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

system_message = "Eres una asistente útil que responde en formato markdown"

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

def stream_gemini(prompt):
    result = google.generativeai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message
    )
    messages=[
            {"role": "user", "parts": prompt}]
    response = ""
    stream = result.generate_content(messages, stream=True)
    for chunk in stream:
        response += chunk.text or ""
        yield response

def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model == "Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result




OpenAI API Key existe y empieza por sk-proj-
Anthropic API Key existe y empieza por sk-ant-
Google API Key existe y empieza por AIzaSyCI


In [10]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Tu mensaje:"), gr.Dropdown(["GPT", "Claude", "Gemini"], label="Selecciona un modelo:", value="GPT")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
